#### Partition functions

In [1]:
# Configuration properties of Apache Spark
#sc.stop()
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp, from_unixtime

APP_NAME = 'pyspark_python'
MASTER = 'local[*]'

conf = SparkConf().setAppName(APP_NAME)
conf = conf.setMaster(MASTER)
spark = SparkSession.builder.config(conf = conf).getOrCreate()
sc = spark.sparkContext

In [2]:
%load_ext autoreload
%autoreload 2
# load my own functions
from utils.partitions import *

In [3]:
import pyspark.sql.functions as psf
from pyspark.sql import Window
from datetime import datetime, timedelta
from itertools import product

In [4]:
data = (  # recreate the DataFrame
    (1, datetime(2019, 12, 2, 14, 54, 17), 49.94),
    (1, datetime(2019, 12, 3, 8, 58, 39), 50.49),
    (1, datetime(2019, 12, 6, 10, 44, 1), 50.24),
    (2, datetime(2019, 12, 2, 8, 58, 39), 62.32),
    (2, datetime(2019, 12, 4, 10, 44, 1), 65.64))
df = spark.createDataFrame(data, schema=("person", "timestamp", "weight"))
df.show()

+------+-------------------+------+
|person|          timestamp|weight|
+------+-------------------+------+
|     1|2019-12-02 14:54:17| 49.94|
|     1|2019-12-03 08:58:39| 50.49|
|     1|2019-12-06 10:44:01| 50.24|
|     2|2019-12-02 08:58:39| 62.32|
|     2|2019-12-04 10:44:01| 65.64|
+------+-------------------+------+



In [5]:
df = df.withColumn('partition_id', create_partitions_from_df('timestamp'))
df.show()

+------+-------------------+------+------------+
|person|          timestamp|weight|partition_id|
+------+-------------------+------+------------+
|     1|2019-12-02 14:54:17| 49.94|    20191231|
|     1|2019-12-03 08:58:39| 50.49|    20191231|
|     1|2019-12-06 10:44:01| 50.24|    20191231|
|     2|2019-12-02 08:58:39| 62.32|    20191231|
|     2|2019-12-04 10:44:01| 65.64|    20191231|
+------+-------------------+------+------------+



Sustract and show de the first day of the month

In [9]:
data_depth_months = 12
sustract_partition('20191231', data_depth_months)

'20181231'

In [10]:
init_day_partition('20191231')

'20191230'

In [20]:
partition_init           = datetime.strftime(datetime.strptime('20191231', '%Y%m%d') -
                                   relativedelta(months=data_depth_months),
                                   '%Y%m%d'
                                  )                          
partition_init

'20181231'

In [18]:
df = df.withColumn('partition_id', init_partition_df('partition_id', 12))
df.show()

ValueError: time data 'partition_id' does not match format 'yyyyMMdd'